# MOVIE RECOMMENDATION SYSTEM

IMPORTING PACKAGES

In [42]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [25]:
column_names = ['MovieID', 'YearOfRelease', 'MovieName']

movie = pd.read_excel('/content/movie_info.xlsx',
                      header=None,
                      names=column_names,
                      usecols="A:C"

                       )
user = pd.read_csv('/content/user_info.txt',
                   names=['MovieID', 'Rating', 'Date'])

num_users = 50
user['CustomerID'] = ['U' + str(i % num_users + 1) for i in range(len(user))]



/tmp/ipython-input-2852705015.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  user = pd.read_csv('/content/user_info.txt',


In [26]:
movie

,MovieID,YearOfRelease,MovieName
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [27]:
user

,MovieID,Rating,Date,CustomerID
0,1488844,3.0,2005-09-06,U1
1,822109,5.0,2005-05-13,U2
2,885013,4.0,2005-10-19,U3
3,30878,4.0,2005-12-26,U4
4,823519,3.0,2004-05-03,U5
...,...,...,...,...
3039476,1151806,5,2004-05-24,U27
3039477,259227,4,2004-07-06,U28
3039478,1706056,004-02-17,NaN,U29
3039479,NaN,4,2004-07-06,U30


DATA CLEANING

In [28]:
movie.drop_duplicates(inplace=True)
movie
user.drop_duplicates(inplace=True)
user


,MovieID,Rating,Date,CustomerID
0,1488844,3.0,2005-09-06,U1
1,822109,5.0,2005-05-13,U2
2,885013,4.0,2005-10-19,U3
3,30878,4.0,2005-12-26,U4
4,823519,3.0,2004-05-03,U5
...,...,...,...,...
3039476,1151806,5,2004-05-24,U27
3039477,259227,4,2004-07-06,U28
3039478,1706056,004-02-17,NaN,U29
3039479,NaN,4,2004-07-06,U30


In [29]:
print(movie.isnull().sum());print(" ")
print(user.isnull().sum())

MovieID          0
YearOfRelease    7
MovieName        0
dtype: int64
 
MovieID         1
Rating        571
Date          572
CustomerID      0
dtype: int64


In [30]:
user['MovieID'].dropna(inplace=True)
user['Rating'].dropna(inplace=True)

user.isnull().sum()
user['Date']=pd.to_datetime(user['Date'],errors='coerce')
user

,MovieID,Rating,Date,CustomerID
0,1488844,3.0,2005-09-06,U1
1,822109,5.0,2005-05-13,U2
2,885013,4.0,2005-10-19,U3
3,30878,4.0,2005-12-26,U4
4,823519,3.0,2004-05-03,U5
...,...,...,...,...
3039476,1151806,5,2004-05-24,U27
3039477,259227,4,2004-07-06,U28
3039478,1706056,004-02-17,NaT,U29
3039479,NaN,4,2004-07-06,U30


In [46]:
user.isnull().sum()

,0
MovieID,0
Rating,0
Date,0
CustomerID,0


In [47]:
movie.isnull().sum()

,0
MovieID,0
YearOfRelease,0
MovieName,0


In [33]:
movie['YearOfRelease'].fillna(movie['YearOfRelease'].mean(), inplace=True)
movie.isnull().sum()

/tmp/ipython-input-1578989399.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movie['YearOfRelease'].fillna(movie['YearOfRelease'].mean(), inplace=True)


,0
MovieID,0
YearOfRelease,0
MovieName,0


In [36]:
user['MovieID'] = pd.to_numeric(user['MovieID'], errors='coerce')
user['Rating'] = pd.to_numeric(user['Rating'], errors='coerce')
user = user.dropna(subset=['MovieID', 'Rating'])


user['MovieID'] = user['MovieID'].astype(int)
user['Rating'] = user['Rating'].astype(int)



/tmp/ipython-input-327147059.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['MovieID'] = user['MovieID'].astype(int)
/tmp/ipython-input-327147059.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['Rating'] = user['Rating'].astype(int)


DATA VIEWING

In [37]:
user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3021620 entries, 0 to 3039477
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   MovieID     int64         
 1   Rating      int64         
 2   Date        datetime64[ns]
 3   CustomerID  object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 115.3+ MB


In [38]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17770 entries, 0 to 17769
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MovieID        17770 non-null  int64  
 1   YearOfRelease  17770 non-null  float64
 2   MovieName      17770 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 416.6+ KB


In [49]:
print(movie.head())
print(user.head())
print(movie.tail())
print(user.tail())

   MovieID  YearOfRelease                     MovieName
0        1         2003.0               Dinosaur Planet
1        2         2004.0    Isle of Man TT 2004 Review
2        3         1997.0                     Character
3        4         1994.0  Paula Abdul's Get Up & Dance
4        5         2004.0      The Rise and Fall of ECW
   MovieID  Rating       Date CustomerID
0  1488844       3 2005-09-06         U1
1   822109       5 2005-05-13         U2
2   885013       4 2005-10-19         U3
3    30878       4 2005-12-26         U4
4   823519       3 2004-05-03         U5
       MovieID  YearOfRelease  \
17765    17766         2002.0   
17766    17767         2004.0   
17767    17768         2000.0   
17768    17769         2003.0   
17769    17770         2003.0   

                                               MovieName  
17765  Where the Wild Things Are and Other Maurice Se...  
17766                  Fidel Castro: American Experience  
17767                                     

DATA SLICING

In [48]:
print(user[user['Rating']>=4])
print(user[user['Rating']<=2])

         MovieID  Rating       Date CustomerID
1         822109       5 2005-05-13         U2
2         885013       4 2005-10-19         U3
3          30878       4 2005-12-26         U4
6         124105       4 2004-08-05         U7
8        1842128       4 2004-05-09         U9
...          ...     ...        ...        ...
3039471  2590295       4 2004-07-18        U22
3039474  1955770       4 2004-08-21        U25
3039475  2419060       4 2004-08-12        U26
3039476  1151806       5 2004-05-24        U27
3039477   259227       4 2004-07-06        U28

[1733931 rows x 4 columns]
         MovieID  Rating       Date CustomerID
36        525356       2 2004-07-11        U37
39       2421815       2 2004-02-26        U40
40       1009622       1 2005-01-19        U41
41       1481961       2 2005-05-24        U42
50       1578279       1 2005-05-19         U1
...          ...     ...        ...        ...
3039427  2186409       2 2004-02-24        U28
3039428  1961875       2 2004-02

DATA MERGING

In [41]:
MOV_USER = pd.merge(user, movie, on="MovieID")
MOV_USER['Date'] = pd.to_datetime(MOV_USER['Date'], errors='coerce')
MOV_USER

,MovieID,Rating,Date,CustomerID,YearOfRelease,MovieName
0,2442,3,2004-04-14,U14,1972.0,Godzilla vs. Gigan
1,4326,4,2005-10-29,U13,1995.0,Ed Sullivan: Rock 'n' Roll Revolution
2,14756,4,2005-12-27,U37,1987.0,White Water Summer
3,3321,3,2005-09-27,U26,1999.0,In Dreams
4,16272,4,2005-01-20,U12,2000.0,Crouching Tiger
...,...,...,...,...,...,...
20613,12147,4,2002-01-16,U22,1954.0,It Should Happen to You
20614,3295,5,2002-01-28,U47,1988.0,Action Jackson
20615,2878,5,2004-04-29,U49,2002.0,Now and Then
20616,5916,3,2004-06-16,U41,2002.0,Geobreeders: Breakthrough


In [45]:
pivot_table = MOV_USER.pivot_table(index='CustomerID',
                                    columns='MovieID',
                                    values='Rating').fillna(0)


sparse_matrix = csr_matrix(pivot_table.values)
similarity = cosine_similarity(sparse_matrix)
similarity_df = pd.DataFrame(similarity,
                             index=pivot_table.index,
                             columns=pivot_table.index)


In [51]:
def get_recom(user_id, top_n=15):

    similar_users = similarity_df[user_id].sort_values(ascending=False)[1:top_n+1].index

    recom_movies = (
        MOV_USER[MOV_USER['CustomerID'].isin(similar_users)]
        .groupby('MovieID')['Rating']
        .mean()
        .sort_values(ascending=False)
        .head(top_n)
        .index
        .tolist()
    )

    recom = movie[movie['MovieID'].isin(recom_movies)][
        ['MovieID', 'MovieName', 'YearOfRelease']
    ]
    return recom.to_dict(orient='records')

output = {
    "RecommendationsForU1": get_recom("U1", top_n=15)
}


In [52]:
with open("RA2512052010042.json", "w") as f:
    json.dump(output, f, indent=4)

print("Collaborative Filtering completed.")

Collaborative Filtering completed.


Recommendation system completed
